In [6]:
from pathlib import Path
import json, pandas as pd

ROOT = Path().resolve()
RAW = ROOT / "D:/project=2/Agrigpt/data_csv/raw"
INTERIM = ROOT / "D:/project=2/Agrigpt/notebook/data/interim"
REPORTS = ROOT / "D:/project=2/Agrigpt/reports"

print("ROOT:", ROOT)
list(INTERIM.glob("*.txt"))


ROOT: D:\project=2\Agrigpt\notebook


[WindowsPath('D:/project=2/Agrigpt/notebook/data/interim/bangladesh_agri_clean.txt'),
 WindowsPath('D:/project=2/Agrigpt/notebook/data/interim/spas_bd_clean.txt')]

In [2]:
import pandas as pd

raw_ba = pd.read_csv(next((RAW/"bangladesh_agri").glob("*.csv")))
with open(INTERIM/"bangladesh_agri_clean.txt", "r", encoding="utf-8") as f:
    after_lines = [l.strip() for l in f.readlines() if l.strip()]

print("Before rows:", len(raw_ba))
print("After lines:", len(after_lines))
print("\nRaw sample row:\n", raw_ba.head(1).T)
print("\nAfter sample line:\n", after_lines[0])


Before rows: 103
After lines: 103

Raw sample row:
                                                   0
Season                                     Kharif 1
Transplant                                    March
Growth                                April to july
Harvest                                         Aug
Products name          Aus Rice(Local: Broadcasted)
Crops Type                                  Cereals
Max Temp                                       40.0
Min Temp                                       12.0
Max Relative Humidity                            60
Min Relative Humidity                            85
Country                                  Bangladesh

After sample line:
 ফসল: Aus Rice(Local: Broadcasted) | মৌসুম: Kharif 1 | রোপণ: March | বৃদ্ধি: April to july | কাটাই: Aug | তাপমাত্রা: 12.0–40.0°C | আপেক্ষিক আর্দ্রতা: 85–60% | দেশ: Bangladesh


In [3]:
raw_sp = pd.read_csv(next((RAW/"spas_bd").glob("SPAS-Dataset-BD*.csv")))
with open(INTERIM/"spas_bd_clean.txt", "r", encoding="utf-8") as f:
    sp_lines = [l.strip() for l in f.readlines() if l.strip()]

print("Before rows:", len(raw_sp))
print("After lines:", len(sp_lines))
print("\nRaw sample row:\n", raw_sp.head(1).T)
print("\nAfter sample line:\n", sp_lines[0])


Before rows: 4608
After lines: 200

Raw sample row:
                                   0
Area                         177321
AP Ratio               0.8510272331
District                   Bagerhat
Season                     Kharif 2
Avg Temp                       26.0
Avg Humidity                   72.5
Crop Name                      Aman
Transplant                     June
Growth                 July to  Oct
Harvest                  Nov to Dec
Production                   150905
Max Temp                         40
Min Temp                       12.0
Max Relative Humidity            60
Min Relative Humidity            85

After sample line:
 ফসল: Aman | জেলা: Bagerhat | মৌসুম: Kharif 2 | রোপণ: June | বৃদ্ধি: July to Oct | কাটাই: Nov to Dec | গড় তাপমাত্রা: 26.0°C | তাপমাত্রা: 12.0–40°C | গড় আর্দ্রতা: 72.5% | আপেক্ষিক আর্দ্রতা: 85–60% | উৎপাদন: 150905


In [4]:
def peek_jsonl(p):
    return [json.loads(l) for l in p.read_text(encoding="utf-8").splitlines()[:3]]

peek_jsonl(INTERIM/"bangladesh_agri_clean.jsonl"), peek_jsonl(INTERIM/"spas_bd_clean.jsonl")


([{'source': 'bangladesh_agri',
   'doc_id': 'doc_51c8bb4f2c18',
   'line_id': 'chunk_doc_51c8bb4f2c18_0000',
   'section_path': 'Aus Rice(Local: Broadcasted) > Season > Kharif 1',
   'language': 'bn',
   'title': 'Bangladesh Agricultural Dataset (104 crops)',
   'created_at': '2025-09-13T09:04:50.480932Z',
   'fields': {'crop_name': 'Aus Rice(Local: Broadcasted)',
    'season': 'Kharif 1',
    'transplant': 'March',
    'growth': 'April to july',
    'harvest': 'Aug',
    'min_temp_c': 12.0,
    'max_temp_c': 40.0,
    'min_rh': 85,
    'max_rh': 60,
    'country': 'Bangladesh'}},
  {'source': 'bangladesh_agri',
   'doc_id': 'doc_51c8bb4f2c18',
   'line_id': 'chunk_doc_51c8bb4f2c18_0001',
   'section_path': 'Aus Rice(HYV: Transplant) > Season > Kharif 1',
   'language': 'bn',
   'title': 'Bangladesh Agricultural Dataset (104 crops)',
   'created_at': '2025-09-13T09:04:50.480932Z',
   'fields': {'crop_name': 'Aus Rice(HYV: Transplant)',
    'season': 'Kharif 1',
    'transplant': 'Marc

In [7]:
pd.read_csv(REPORTS/"preprocess_quality_report.csv")


,file,exists,lines,avg_tokens,p95_tokens,max_tokens
0,bangladesh_agri_clean.txt,True,103,28.91,31,31
1,spas_bd_clean.txt,True,200,39.30,40,41


In [8]:
# Example threshold: at least 95% of rows retained as lines (no giant drops)
ba_before = len(raw_ba)
sp_before = len(raw_sp)
ba_after = len(open(INTERIM/"bangladesh_agri_clean.txt", encoding="utf-8").read().splitlines())
sp_after = len(open(INTERIM/"spas_bd_clean.txt", encoding="utf-8").read().splitlines())

retention = pd.DataFrame([
    {"source":"bangladesh_agri","retention": round(ba_after/ba_before*100, 2)},
    {"source":"spas_bd","retention": round(sp_after/sp_before*100, 2)},
])
retention


,source,retention
0,bangladesh_agri,100.00
1,spas_bd,4.34


In [10]:
from pathlib import Path
import pandas as pd

RAW = Path(r"D:/project=2/Agrigpt/data_csv/raw")
INTERIM = Path(r"D:/project=2/Agrigpt/notebook/data/interim")

raw_ba = pd.read_csv(next((RAW/"bangladesh_agri").glob("*.csv")))
raw_sp = pd.read_csv(next((RAW/"spas_bd").glob("SPAS-Dataset-BD*.csv")))

after_ba = sum(1 for _ in open(INTERIM/"bangladesh_agri_clean.txt", encoding="utf-8"))
after_sp = sum(1 for _ in open(INTERIM/"spas_bd_clean.txt", encoding="utf-8"))

pd.DataFrame([
    {"source":"bangladesh_agri","retention": round(after_ba/len(raw_ba)*100, 2)},
    {"source":"spas_bd","retention": round(after_sp/len(raw_sp)*100, 2)},
])


,source,retention
0,bangladesh_agri,100.0
1,spas_bd,100.0
